In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random
import math
import itertools
import pprint

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin

drive_path = '/content/drive/MyDrive/Kuliah/Tugas Akhir/Final Project Shared Folder'
data_path = "Dataset/Data Versioning/"
model_path = "Model/ML Model/"
hyperparameter_log_path = "Notebook/Arif's Workspace/Hyperparameter Tuning/Log/Merged"
data_version = "Trained_V2-2.csv"
hyperparameter_log = "V3.csv"
base_url = "https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?CycleBeginYear=2017"
dataset_names = ['Demographics', 'Dietary', 'Examination', 'Laboratory', 'Questionnaire']

# Model Builder

In [2]:
class CNNModel(BaseEstimator, ClassifierMixin):
  def __init__(
      self,
      input_shape,
      output_shape,
      layer_metadata,
      convolution_part=['conv1d'],
      intermediate_part=['flatten'],
      dense_part=['dense'],
      convolution_hyperparameter=[{'filters':16, 'kernel_size':3, 'activation':'relu'}],
      intermediate_hyperparameter=[{}],
      dense_hyperparameter=[{'units':32, 'activation':'relu'}],
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy']
  ):
    self.input_shape = input_shape
    self.layer_metadata = layer_metadata
    self.convolution_part = convolution_part
    self.intermediate_part = intermediate_part
    self.dense_part = dense_part
    self.convolution_hyperparameter = convolution_hyperparameter
    self.intermediate_hyperparameter = intermediate_hyperparameter
    self.dense_hyperparameter = dense_hyperparameter
    self.optimizer = optimizer
    self.model = self.build_model(input_shape, output_shape, convolution_part, intermediate_part, dense_part, convolution_hyperparameter, intermediate_hyperparameter, dense_hyperparameter, optimizer, loss, metrics)



  def build_model(
      self,
      input_shape,
      output_shape,
      convolution_part,
      intermediate_part,
      dense_part,
      convolution_hyperparameter,
      intermediate_hyperparameter,
      dense_hyperparameter,
      optimizer,
      loss,
      metrics
  ):

    def fix_hyperparameter_type(layer, hyperparameter, layer_metadata, warnings=True):
      for layer_info in layer_metadata:
        if(layer == layer_info['layer']):
          for params in layer_info['hyperparameter']:
            val = hyperparameter.get(params['param'], params['default'])
            if(type(val) != params['type']):
              if(warnings): print("Hyperparameter ", params['param'], " Have Invalid Data Type! Using Default..")
              hyperparameter[params['param']] = params['default']
            else:
              hyperparameter[params['param']] = val
        else:
          pass
      return hyperparameter

    model = keras.models.Sequential()
    model.add(keras.layers.Input(input_shape))

    # Convolution Part
    for layer, hyperparameter in zip(convolution_part, convolution_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'conv1d'):
        model.add(keras.layers.Conv1D(filters=hyperparameter['filters'], kernel_size=hyperparameter['kernel_size'], activation=hyperparameter['activation']))
      elif(layer == 'maxpooling1d'):
        model.add(keras.layers.MaxPooling1D(pool_size=hyperparameter['pool_size']))
      elif(layer == 'conv2d'):
        model.add(keras.layers.Conv2D(filters=hyperparameter['filters'], kernel_size=hyperparameter['kernel_size'], activation=hyperparameter['activation']))
      elif(layer == 'maxpooling2d'):
        model.add(keras.layers.MaxPooling2D(pool_size=hyperparameter['pool_size']))
      else:
        print("'Convolution Part' Layer Invalid!")
        return None

    # Intermediate Part
    for layer, hyperparameter in zip(intermediate_part, intermediate_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'flatten'):
        model.add(keras.layers.Flatten())
      else:
        print("'Intermediate Part' Layer Invalid!")
        return None

    # Dense Part
    for layer, hyperparameter in zip(dense_part, dense_hyperparameter):
      hyperparameter = fix_hyperparameter_type(layer, hyperparameter, self.layer_metadata)

      if(layer == 'dense'):
        model.add(keras.layers.Dense(units=hyperparameter['units'], activation=hyperparameter['activation']))
      else:
        print("'Dense Part' Layer Invalid!")
        return None

    model.add(keras.layers.Dense(output_shape, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=optimizer,
                loss=loss,
                metrics=metrics)

    return model

  def fit(self, X_train, y_train, X_test, y_test, epochs, verbose=1):
    self.model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=verbose)

  def score(self, X, y):
    loss, accuracy = self.model.evaluate(X, y, verbose=0)
    loss_inverse = 1/(loss+1e-20)

    return loss_inverse

  def evaluate(self, X, y):
    # Evaluate the model on the test set
    loss, accuracy = self.model.evaluate(X, y, verbose=0)

    print(f"Loss: %.3f%%" % (loss*100) )
    print(f"Accuracy: %.3f%%" % (accuracy*100) )

    return loss, accuracy

  def summary(self):
    self.model.summary()

  def predict(self, X):
    return self.model.predict(X, verbose=0)

In [3]:
layer_metadata = [
  {'layer': 'conv1d', 'hyperparameter': [
      {'param':'filters', 'type':int, 'default':16},
      {'param':'kernel_size', 'type':int, 'default':3},
      {'param':'activation', 'type':str, 'default':'linear'}
    ]
  },
  {'layer': 'maxpooling1d', 'hyperparameter': [
      {'param':'pool_size', 'type':int, 'default':2}
    ]
  },
  {'layer': 'flatten', 'hyperparameter': [

    ]
  },
  {'layer': 'dense', 'hyperparameter': [
      {'param':'units', 'type':int, 'default':8},
      {'param':'activation', 'type':str, 'default':'linear'}
    ]
  }
]

# Check Hyperparameter Log

In [5]:
hp_log = pd.read_csv(os.path.join(drive_path, hyperparameter_log_path+"/"+hyperparameter_log))

hp_log['convolution_hyperparameter'].replace(r"<class '(.*?)'>", r"\1", regex=True)
hp_log['dense_hyperparameter'].replace(r"<class '(.*?)'>", r"\1", regex=True)

hp_log

,convolution_part,convolution_hyperparameter,dense_part,dense_hyperparameter,input_shape,layer_metadata,loss,accuracy,precision,recall,f1score,auc,train_loss,train_accuracy,train_precision,train_recall,train_f1score,train_auc
0,"['maxpooling1d', 'maxpooling1d', 'conv1d', 'co...","[{'pool_size': 2}, {'pool_size': 2}, {'filters...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.199149,0.921386,0.827381,0.500744,0.467545,0.921215,0.176368,0.921386,0.839286,0.469162,0.500744,0.930463
1,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 2, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.302882,0.823025,0.827381,0.500744,0.485084,0.827505,0.289515,0.823025,0.839286,0.486844,0.500744,0.833068
2,"['conv1d', 'maxpooling1d', 'conv1d', 'conv1d']","[{'filters': 2, 'kernel_size': 2, 'activation'...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.096298,0.971032,0.827381,0.500744,0.481466,0.970911,0.031663,0.971032,0.839286,0.482562,0.500744,0.988581
3,"['conv1d', 'conv1d', 'conv1d']","[{'filters': 16, 'kernel_size': 3, 'activation...",['dense'],"[{'units': 4, 'activation': 'relu'}]","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.103321,0.963767,0.827381,0.500744,0.480031,0.964108,0.049681,0.963767,0.839286,0.481159,0.500744,0.980841
4,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 16, 'kernel_size': 2, 'activation...",['dense'],"[{'units': 8, 'activation': 'relu'}]","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.098768,0.965723,0.827381,0.500744,0.479413,0.966139,0.038228,0.965723,0.839286,0.481960,0.500744,0.985780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 16, 'kernel_size': 2, 'activation...","['dense', 'dense']","[{'units': 8, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.123619,0.955570,0.827381,0.500744,0.480977,0.955640,0.067690,0.955570,0.839286,0.483230,0.500744,0.975315
164,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 4, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.125039,0.961624,0.827381,0.500744,0.480735,0.962039,0.054563,0.961624,0.839286,0.482509,0.500744,0.979603
165,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 4, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.097731,0.972057,0.827381,0.500744,0.479868,0.972331,0.025190,0.972057,0.839286,0.481817,0.500744,0.990604
166,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 4, 'kernel_size': 2, 'activation'...","['dense', 'dense']","[{'units': 8, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.110621,0.962370,0.827381,0.500744,0.480793,0.962378,0.065290,0.962370,0.839286,0.481560,0.500744,0.977321


In [6]:
hp_log_sorted = hp_log.copy()
hp_log_sorted = hp_log_sorted.sort_values(by=['accuracy', 'loss', 'train_accuracy', 'train_loss'], ascending=[True, False, True, False])

hp_log_sorted

,convolution_part,convolution_hyperparameter,dense_part,dense_hyperparameter,input_shape,layer_metadata,loss,accuracy,precision,recall,f1score,auc,train_loss,train_accuracy,train_precision,train_recall,train_f1score,train_auc
76,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 2, 'kernel_size': 2, 'activation'...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.562574,0.598733,0.827381,0.500744,0.243348,0.610118,0.551376,0.598733,0.839286,0.251921,0.500744,0.614324
141,"['conv1d', 'conv1d', 'maxpooling1d', 'conv1d']","[{'filters': 8, 'kernel_size': 2, 'activation'...","['dense', 'dense']","[{'units': 4, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.411400,0.722522,0.827381,0.500744,0.499589,0.728601,0.389191,0.722522,0.839286,0.501119,0.500744,0.734851
54,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 8, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.401671,0.725876,0.827381,0.500744,0.499242,0.732096,0.372893,0.725876,0.839286,0.501015,0.500744,0.741923
1,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 2, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.302882,0.823025,0.827381,0.500744,0.485084,0.827505,0.289515,0.823025,0.839286,0.486844,0.500744,0.833068
85,"['maxpooling1d', 'conv1d', 'conv1d', 'conv1d']","[{'pool_size': 2}, {'filters': 8, 'kernel_size...","['dense', 'dense']","[{'units': 4, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.292361,0.830104,0.827381,0.500744,0.487581,0.832068,0.274766,0.830104,0.839286,0.490140,0.500744,0.836693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 4, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.097731,0.972057,0.827381,0.500744,0.479868,0.972331,0.025190,0.972057,0.839286,0.481817,0.500744,0.990604
124,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 4, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.108618,0.972057,0.827381,0.500744,0.483101,0.971822,0.024550,0.972057,0.839286,0.483543,0.500744,0.990701
109,"['conv1d', 'conv1d', 'conv1d', 'conv1d']","[{'filters': 4, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.107867,0.973268,0.827381,0.500744,0.480260,0.973149,0.019614,0.973268,0.839286,0.482366,0.500744,0.992297
45,"['conv1d', 'conv1d', 'conv1d']","[{'filters': 8, 'kernel_size': 3, 'activation'...","['dense', 'dense']","[{'units': 16, 'activation': 'relu'}, {'units'...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.081749,0.973268,0.827381,0.500744,0.483427,0.973330,0.015969,0.973268,0.839286,0.484122,0.500744,0.994087


In [7]:
# Top 5 Model

for i in range(5):
  print("Best Model #", i+1)
  print("-"*10)
  pprint.pprint(hp_log_sorted.iloc[-i-1].to_dict())
  print("-"*10)

Best Model # 1
----------
{'accuracy': 0.9733606576919556,
 'auc': 0.9735129565828856,
 'convolution_hyperparameter': "[{'filters': 12, 'kernel_size': 3, "
                               "'activation': 'relu'}, {'filters': 2, "
                               "'kernel_size': 3, 'activation': 'relu'}, "
                               "{'filters': 8, 'kernel_size': 3, 'activation': "
                               "'relu'}, {'filters': 16, 'kernel_size': 3, "
                               "'activation': 'relu'}]",
 'convolution_part': "['conv1d', 'conv1d', 'conv1d', 'conv1d']",
 'dense_hyperparameter': "[{'units': 16, 'activation': 'relu'}, {'units': 8, "
                         "'activation': 'relu'}]",
 'dense_part': "['dense', 'dense']",
 'f1score': 0.4792205765843391,
 'input_shape': '(85, 1)',
 'layer_metadata': "[{'layer': 'conv1d', 'hyperparameter': [{'param': "
                   "'filters', 'type': <class 'int'>, 'default': 16}, "
                   "{'param': 'kernel_size', 't

In [ ]:
# Bottom 5 Model

for i in range(5):
  print("Worst Model #", i+1)
  print("-"*10)
  pprint.pprint(hp_log_sorted.iloc[i].to_dict())
  print("-"*10)

Worst Model # 1
----------
{'accuracy': 0.7771123945713043,
 'convolution_hyperparameter': "[{'filters': 16, 'kernel_size': 2, "
                               "'activation': 'relu'}, {'filters': 4, "
                               "'kernel_size': 2, 'activation': 'relu'}, "
                               "{'filters': 16, 'kernel_size': 2, "
                               "'activation': 'relu'}]",
 'convolution_part': "['conv1d', 'conv1d', 'conv1d']",
 'dense_hyperparameter': "[{'units': 8, 'activation': 'relu'}, {'units': 2, "
                         "'activation': 'relu'}]",
 'dense_part': "['dense', 'dense']",
 'input_shape': '(85, 1)',
 'layer_metadata': "[{'layer': 'conv1d', 'hyperparameter': [{'param': "
                   "'filters', 'type': <class 'int'>, 'default': 16}, "
                   "{'param': 'kernel_size', 'type': <class 'int'>, 'default': "
                   "3}, {'param': 'activation', 'type': <class 'str'>, "
                   "'default': 'linear'}]}, {'layer':

In [ ]:
hp_log_sorted.set_index('convolution_part')

,convolution_hyperparameter,dense_part,dense_hyperparameter,input_shape,layer_metadata,loss,accuracy,train_loss,train_accuracy
convolution_part,,,,,,,,,
"['conv1d', 'conv1d', 'conv1d']","[{'filters': 16, 'kernel_size': 2, 'activation...","['dense', 'dense']","[{'units': 8, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.395260,0.777112,0.386128,0.780297
"['conv1d', 'conv1d', 'conv1d']","[{'filters': 8, 'kernel_size': 2, 'activation'...","['dense', 'dense']","[{'units': 4, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.394043,0.777112,0.390169,0.779973
"['maxpooling1d', 'conv1d', 'conv1d']","[{'pool_size': 2}, {'filters': 4, 'kernel_size...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.387605,0.777934,0.383692,0.779771
"['conv1d', 'conv1d', 'conv1d']","[{'filters': 12, 'kernel_size': 3, 'activation...","['dense', 'dense']","[{'units': 8, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.385361,0.778258,0.380086,0.779987
"['conv1d', 'conv1d', 'conv1d']","[{'filters': 16, 'kernel_size': 2, 'activation...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.390873,0.778906,0.389107,0.778942
...,...,...,...,...,...,...,...,...,...
"['maxpooling1d', 'conv1d', 'conv1d']","[{'pool_size': 2}, {'filters': 4, 'kernel_size...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.095816,0.978820,0.089106,0.978676
"['maxpooling1d', 'conv1d', 'conv1d']","[{'pool_size': 2}, {'filters': 4, 'kernel_size...","['dense', 'dense']","[{'units': 2, 'activation': 'relu'}, {'units':...","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.095787,0.978820,0.089088,0.978712
"['conv1d', 'conv1d', 'conv1d']","[{'filters': 2, 'kernel_size': 2, 'activation'...",['dense'],"[{'units': 8, 'activation': 'relu'}]","(85, 1)","[{'layer': 'conv1d', 'hyperparameter': [{'para...",0.075142,0.978928,0.063804,0.979396
